## Preparation

In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Function to plot results for a given graph
def plot_experiment_results(df: pd.DataFrame, exp_name: str, graph_name: str, img_folder="Graphics/"):
    # Load experimental results
    df = pd.read_pickle(f"Experiments/{exp_name}.pkl")

    # Grouping and counting fulfillment of conditions
    df["Irrep. fulfilled"] = df["Irrep. fulfilled"].astype(int)
    df["Weak irrep. fulfilled"] = df["Weak irrep. fulfilled"].astype(int)
    condition_counts = df.groupby(["Intervention"]).agg({"Irrep. fulfilled": "sum", "Weak irrep. fulfilled": "sum"}).reset_index()

    # Define unique interventions
    unique_interventions = df["Intervention"].unique()
    num_interventions = len(unique_interventions)

    # Create a figure with dynamic rows based on the number of interventions
    fig, axes = plt.subplots(nrows=max(num_interventions, 2), ncols=2, figsize=(10, 3 * max(num_interventions, 2)))

    # Flatten axes array if necessary
    if num_interventions == 1:
        axes = axes.reshape(-1, 2)  # Ensure it's always 2 columns

    # **1️⃣ Left Column: Graph Image**
    graph_img_path = os.path.join(f"Graphics/{graph_name}.png")
    if os.path.exists(graph_img_path):
        img = plt.imread(graph_img_path)
        axes[0, 0].imshow(img)
        axes[0, 0].axis("off")
        axes[0, 0].set_title(f"Graph: {graph_name}")

    # **2️⃣ Left Column: Table with Condition Fulfillment**
    percentage_data = condition_counts.copy()
    percentage_data["Irrep. fulfilled"] = (percentage_data["Irrep. fulfilled"] / 1000).round(1)
    percentage_data["Weak irrep. fulfilled"] = (percentage_data["Weak irrep. fulfilled"] / 1000).round(1)

    # Hide axes for the table
    axes[1, 0].axis("tight")
    axes[1, 0].axis("off")

    # Create the first table (absolute counts)
    table1 = axes[1, 0].table(cellText=condition_counts.values, 
                            colLabels=condition_counts.columns, 
                            rowLabels=[''] * len(condition_counts),  # Hide row labels
                            cellLoc='center', loc='center', bbox=[0, 0.55, 1, 0.3])

    # Create the second table (percentage values)
    table2 = axes[1, 0].table(cellText=percentage_data.values, 
                            colLabels=percentage_data.columns, 
                            rowLabels=[''] * len(percentage_data),  
                            cellLoc='center', loc='center', bbox=[0, 0.05, 1, 0.3])

    # Set title for table
    axes[1, 0].set_title("Condition Fulfillment Counts & Percentages")

    # **3️⃣ Right Column: Scatter Plots for Each Intervention**
    for i, intervention in enumerate(unique_interventions):
        subset = df[df["Intervention"] == intervention]  # Select data for this intervention

        # Add experiment index as x-axis (0, 1, 2, ..., len(subset)-1)
        subset = subset.reset_index()

        # Scatter plot for this intervention
        sns.scatterplot(data=subset, x=subset.index, y="Performance lasso", ax=axes[i, 1])

        # Labels and title
        axes[i, 1].set_title(f"Intervention {intervention}")
        axes[i, 1].set_xlabel("Experiment Number")
        axes[i, 1].set_ylabel("Frobenius Distance" if i == 0 else "")  # Only first plot has y-label

    # **Adjust layout for better spacing**
    plt.tight_layout()
    plt.savefig(f"Evaluation/{exp_name}.png")
    plt.show()

## Evaluation